In [ ]:
import _pickle as cPickle
f = open('data_set2.pkl','rb')
train_set, test_set, dicts = cPickle.load(f,encoding='utf-8')

train_lex, train_y, train_z = train_set
test_lex,  test_y, test_z  = test_set

MAX_LENGTH = len(max(train_lex, key=len))


from keras.preprocessing.sequence import pad_sequences
train_sentences_X = pad_sequences(train_lex,maxlen=MAX_LENGTH,padding='post')
train_tags_y = pad_sequences(train_y,maxlen=MAX_LENGTH,padding='post')

test_sentences_X = pad_sequences(test_lex,maxlen=MAX_LENGTH,padding='post')
test_tags_y = pad_sequences(test_y,maxlen=MAX_LENGTH,padding='post')

train_tags_z = pad_sequences(train_z,maxlen=MAX_LENGTH,padding='post')
test_tags_z = pad_sequences(test_z,maxlen=MAX_LENGTH,padding='post')


import numpy as np
def to_categorical(sequences,categories):
    cat_sequences = []
    for s in sequences:
        cats = []
        for item in s:
            cats.append([item])
        cat_sequences.append(cats)
    return np.array(cat_sequences)

def to_categorical2(sequences, categories):
    cat_sequences = []
    for s in sequences:
        cats = []
        for item in s:
            cats.append(np.zeros(categories))
            cats[-1][item] = 1.0
        cat_sequences.append(cats)
    return np.array(cat_sequences)


from keras.models import Sequential
from keras.layers import Dense, LSTM, InputLayer, Bidirectional, TimeDistributed, Embedding, Activation, Dropout, Input
from keras.optimizers import Adam

from keras import backend as K
 
def ignore_class_accuracy(to_ignore=0):
    def ignore_accuracy(y_true, y_pred):
        y_true_class = K.argmax(y_true, axis=-1)
        y_pred_class = K.argmax(y_pred, axis=-1)
        ignore_mask = K.cast(K.not_equal(y_pred_class, to_ignore), 'float32')
        matches = K.cast(K.equal(y_true_class, y_pred_class), 'float32') * ignore_mask
        accuracy = K.sum(matches) / K.maximum(K.sum(ignore_mask), 1)
        return accuracy
    return ignore_accuracy


from keras.models import Model
from keras.optimizers import Adam
from tensorflow import keras
from tensorflow.keras import layers
from keras.models import Model

inputLayer = Input(shape=(MAX_LENGTH,),dtype='float32')
embedding = Embedding(len(dicts['words2idx'].keys())+1,128,mask_zero=True)(inputLayer)
hiddenLayer1 = Bidirectional(LSTM(128,return_sequences=True))(embedding)
dropoutLayer = Dropout(0.5)(hiddenLayer1)
hiddenLayer2 = Bidirectional(LSTM(128,return_sequences=True))(dropoutLayer)
keyphraseDenseLayer = TimeDistributed(Dense(6,activation='softmax'),name='keyphrase')(hiddenLayer2)
model1 = Model(inputLayer, keyphraseDenseLayer)


model1 = Model(inputLayer, keyphraseDenseLayer)

model1.compile(loss = 'categorical_crossentropy',
             optimizer = Adam(0.01),
             metrics = ['accuracy',ignore_class_accuracy(0)])

f = model1.fit(train_sentences_X,to_categorical2(train_tags_z,6),batch_size=128,epochs=1,validation_split=0.2,shuffle=True)


In [ ]:
scores = model1.evaluate(test_sentences_X, to_categorical2(test_tags_z, 6))
print(scores)